# Estimator minimal working examle (CNN)

In [ ]:
import tensorflow as tf
import numpy as np
train, test = tf.keras.datasets.mnist.load_data()

In [ ]:
train_data = train[0]/255
test_data = test[0]/255

## Definition of Feature Columns

In [ ]:
feature_columns = []
image_data = tf.feature_column.numeric_column("input_x", shape=[28, 28])
feature_columns.append(image_data)

## Building the Classifier aka "Model"

We are using a premade estimator (DNNClassifier) here. Using a premade estimator enables us to create our model with just 7 LOC, it doesnt get easier than then (Well probably it will...). I know, a Convolutional NN would be the better choice of a weapon, but just to show you the power of the predefined estimators...

In [ ]:
classifier = tf.estimator.DNNClassifier(
 feature_columns=feature_columns,
 optimizer=tf.train.AdamOptimizer(1e-4), #Default is Adagrad, why not using Adam with a learning rate of 0.0001 ...
 hidden_units=[256, 32], #Number of values defines the number of layers in the NN. Values itself define number of neurons in the layer.
 n_classes=10,
 dropout=0.5
)

## Define the training function

In [ ]:
def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(({"input_x": features}, labels.astype(np.int32)))
    dataset = dataset.shuffle(100000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [ ]:
classifier.train(input_fn=lambda:train_input_fn(train_data,train[1],50), steps=50000)

## Define Test

In [ ]:
def test_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(({"input_x": features}, labels.astype(np.int32)))
    dataset = dataset.batch(batch_size)
    
    return dataset.make_one_shot_iterator().get_next()

In [ ]:
accuracy_score = classifier.evaluate(input_fn=lambda:test_input_fn(test_data,test[1],50))
print(accuracy_score)